In [24]:
%load_ext autoreload
%autoreload 2
import clipper_manager as cl
import os
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn import svm
# import sklearn.ensemble.RandomForestClassifier as RFC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.externals import joblib
import sys


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Train a Scikit-Learn model

In [2]:
def load_digits(digits_location, digits_filename = "train.data", norm=True):
    digits_path = digits_location + "/" + digits_filename
    print("Source file: %s" % digits_path)
    df = pd.read_csv(digits_path, sep=",", header=None)
    data = df.values
    print("Number of image files: %d" % len(data))
    y = data[:,0]
    X = data[:,1:]
    Z = X
    if norm:
        mu = np.mean(X,0)
        sigma = np.var(X,0)
        Z = (X - mu) / np.array([np.sqrt(z) if z > 0 else 1. for z in sigma])
    label = 3
    my_y = [1. if i == label else -1.0 for i in y]
    return (Z, my_y)

label = 3

fname = "log_regression"
path = '/tmp/%s/%s.pkl' % (fname, fname)
print("Training logistic regression")
X, y = load_digits(os.path.expanduser("~/model-serving/data/mnist_data"))
model = lm.LogisticRegression()
model.fit(X, y)
try:
    os.mkdir('/tmp/%s' % fname)
except OSError:
    print("directory already exists. Might overwrite existing file")
joblib.dump(model, path) 
print("Done training logistic regression model")



Training logistic regression
Source file: /Users/crankshaw/model-serving/data/mnist_data/train.data
Number of image files: 60000
Done training logistic regression model


In [32]:
clipper.stop_all()
ec2_host = "ec2-54-158-88-117.compute-1.amazonaws.com"
user = "ubuntu"
key = os.path.expanduser("~/.ssh/aws_rsa")

clipper = cl.Cluster(ec2_host, user, key)
clipper.start_clipper()

[ec2-54-158-88-117.compute-1.amazonaws.com] sudo: docker stop $(docker ps -a -q)
[ec2-54-158-88-117.compute-1.amazonaws.com] out: f48244d1222a
[ec2-54-158-88-117.compute-1.amazonaws.com] out: f84a36ad95bd
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 90ea7dc7d3ad
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 

[ec2-54-158-88-117.compute-1.amazonaws.com] sudo: docker rm $(docker ps -a -q)
[ec2-54-158-88-117.compute-1.amazonaws.com] out: f48244d1222a
[ec2-54-158-88-117.compute-1.amazonaws.com] out: f84a36ad95bd
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 90ea7dc7d3ad
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 

ubuntu
[]
[ec2-54-158-88-117.compute-1.amazonaws.com] sudo: docker ps
[ec2-54-158-88-117.compute-1.amazonaws.com] out: CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 

[ec2-54-158-88-117.compute-1.amazonaws.com] sudo: docker network creat

In [33]:
clipper.add_sklearn_model("sklearn_logistic_regression", "/tmp/log_regression")

/tmp/clipper-models/sklearn_logistic_regression/1
[ec2-54-158-88-117.compute-1.amazonaws.com] run: mkdir -p /tmp/clipper-models/sklearn_logistic_regression/1
[ec2-54-158-88-117.compute-1.amazonaws.com] put: /tmp/log_regression/log_regression.pkl -> /tmp/clipper-models/sklearn_logistic_regression/1/./log_regression/log_regression.pkl
[ec2-54-158-88-117.compute-1.amazonaws.com] put: /tmp/log_regression/log_regression.pkl_01.npy -> /tmp/clipper-models/sklearn_logistic_regression/1/./log_regression/log_regression.pkl_01.npy
[ec2-54-158-88-117.compute-1.amazonaws.com] put: /tmp/log_regression/log_regression.pkl_02.npy -> /tmp/clipper-models/sklearn_logistic_regression/1/./log_regression/log_regression.pkl_02.npy
[ec2-54-158-88-117.compute-1.amazonaws.com] put: /tmp/log_regression/log_regression.pkl_03.npy -> /tmp/clipper-models/sklearn_logistic_regression/1/./log_regression/log_regression.pkl_03.npy
[ec2-54-158-88-117.compute-1.amazonaws.com] put: /tmp/log_regression/log_regression.pkl_04.n


[ec2-54-158-88-117.compute-1.amazonaws.com] sudo: docker run -d --network=clipper_nw --name sklearn_logistic_regression_v1_r0 -v /tmp/clipper-models/sklearn_logistic_regression/1/log_regression:/model:ro dcrankshaw/clipper-sklearn-mw
[ec2-54-158-88-117.compute-1.amazonaws.com] out: c334302189784202bbaf95d94796363f25d64dde5537464fdc0f0088226cf82a
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 



In [34]:
clipper.get_metrics()

{
    "meters": [
        {
            "rate": 0.0, 
            "name": "prediction_thruput", 
            "unit": "events per second"
        }, 
        {
            "rate": 0.0, 
            "name": "update_thruput", 
            "unit": "events per second"
        }, 
        {
            "rate": 0.0, 
            "name": "sklearn_logistic_regression_model_thruput", 
            "unit": "events per second"
        }
    ], 
    "histograms": [
        {
            "std": 0.0, 
            "p99": 0.0, 
            "name": "prediction_latency", 
            "min": 0, 
            "max": 0, 
            "p95": 0.0, 
            "p50": 0.0, 
            "mean": 0.0
        }, 
        {
            "std": 0.0, 
            "p99": 0.0, 
            "name": "in_time_predictions", 
            "min": 0, 
            "max": 0, 
            "p95": 0.0, 
            "p50": 0.0, 
            "mean": 0.0
        }, 
        {
            "std": 0.0, 
            "p99": 0.0, 
            "

In [ ]:
# clipper.start_clipper(config="/Users/crankshaw/clipper/clipper_server/conf/test.toml")
clipper.start_clipper()

In [4]:
clipper.stop_all()

[ec2-54-158-88-117.compute-1.amazonaws.com] sudo: docker stop $(docker ps -a -q)
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 2391a5d5f5fd
[ec2-54-158-88-117.compute-1.amazonaws.com] out: fb5e3300333a
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 

[ec2-54-158-88-117.compute-1.amazonaws.com] sudo: docker rm $(docker ps -a -q)
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 2391a5d5f5fd
[ec2-54-158-88-117.compute-1.amazonaws.com] out: fb5e3300333a
[ec2-54-158-88-117.compute-1.amazonaws.com] out: 



In [ ]:
clipper.get_metrics()